In [ ]:
import pandas as pd
import json
import glob, os
from sfem import SFEM

In [ ]:
def metrics(df, df2):
    
    #score
    score = 0
    for row in range(0,df.shape[0]):
        if df['A'].iloc[row] == "cooperate" and df['B'].iloc[row] == "cooperate":
            score +=1
        if df['A'].iloc[row] == "cooperate" and df['B'].iloc[row] == "defect":
            score +=5
        if df['A'].iloc[row] == "defect" and df['B'].iloc[row] == "defect":
            score +=3

    #total_score
    total_score = 0
    for row in range(0,df.shape[0]):
        if df['A'].iloc[row] == "cooperate" and df['B'].iloc[row] == "cooperate":
            total_score +=2
        if df['A'].iloc[row] == "cooperate" and df['B'].iloc[row] == "defect":
            total_score +=5
        if df['A'].iloc[row] == "defect" and df['B'].iloc[row] == "defect":
            total_score +=6
        if df['A'].iloc[row] == "defect" and df['B'].iloc[row] == "cooperate":
            total_score +=5

    #defect rate
    defect_rate = df['A'][df['A']=="defect"].count()/df['A'].count()

    #Reciprocity rate
    rr_count = 0
    rr_invalid = 0
    for row in range(1,df.shape[0]):
        if df['A'].iloc[row] != "None" and df['B'].iloc[row-1] != "None":
            if df['A'].iloc[row] == df['B'].iloc[row-1]:
                rr_count +=1
        else:
            rr_invalid +=1
    rr_percentage = rr_count/(df.shape[0]-1)

    #Forgiveness rate based off paper "Nicer Than Humans: How do Large Language Models Behave in the Prisoner’s Dilemma?"
    f_count = 0
    opp_betray_count = 0
    penalty = 0
    f_invalid = 0
    vengeful = False
    for row in range(1,df.shape[0]):
        if df['A'].iloc[row] != "None" and df['A'].iloc[row-1] != "None" and df['B'].iloc[row-1] != "None":
            if df['B'].iloc[row-1] == "defect":
                opp_betray_count += 1
                if df['A'].iloc[row] == "cooperate":
                    f_count += 1
                    vengeful = False
            if row > 1:
                if df['B'].iloc[row-2] != "None":
                    if df['B'].iloc[row-2] == "defect" and df['B'].iloc[row-1] == "cooperate":
                        if df['A'].iloc[row] == "defect" and df['A'].iloc[row-1] == "defect":
                            penalty += 1
                            vengeful = True
                        else:
                            vengeful = False
                    else:
                        if vengeful and df['A'].iloc[row] == "defect":
                            penalty += 1
                        else:
                            vengeful = False
                else:
                    f_invalid +=1
        else:
            f_invalid +=1

    if opp_betray_count == 0:
        f_percentage = -1
    else:
        f_percentage = f_count / (opp_betray_count + penalty)

    #Forgiveness rate based on  we only count forgiveness if the agent plays betrays after opponent betrays and then revert back to cooperation if the opponent seeks forgiveness (switches to cooperates). Penalty term remains.
    # f_count = 0
    # opp_betray_count = 0
    # penalty = 0
    # f_invalid = 0
    # vengeful = False
    # for row in range(1,df.shape[0]):
    #     if df['invalid'].iloc[row] == False and df['invalid'].iloc[row-1] == False:
    #         if df['betray'].iloc[row-1] == True:
    #             opp_betray_count += 1
    #         if row > 1:
    #             if df['betray'].iloc[row-2] == True and df['betray'].iloc[row-1] == False:
    #                 if df['betray'].iloc[row] == True and df['betray'].iloc[row-1] == True:
    #                     penalty += 1
    #                     vengeful = True
    #                 elif df['betray'].iloc[row] == True and df['betray'].iloc[row-1] == False:
    #                     f_count += 1
    #                     vengeful = False
    #         else:
    #             if vengeful and df['betray'].iloc[row] == True:
    #                 penalty += 1
    #             else:
    #                 vengeful = False
    #     else:
    #         f_invalid +=1

    # f_percentage = f_count / (opp_betray_count + penalty)

    #Exploitability
    e_count = 0
    opp_betray_count = 0
    e_invalid = 0
    for row in range(1,df.shape[0]):
        if df['A'].iloc[row] != "None" and df['B'].iloc[row-1] != "None":
            if df['B'].iloc[row-1] == "defect":
                opp_betray_count += 1
                if df['A'].iloc[row] == "cooperate":
                    e_count += 1
        else:
            e_invalid +=1
            
    if opp_betray_count == 0:
        e_percentage = -1
    else:
        e_percentage = e_count / opp_betray_count

    #Retaliatory
    r_count = 0
    opp_betray_count = 0
    r_invalid = 0
    for row in range(1,df.shape[0]):
        if df['A'].iloc[row] != "None" and df['A'].iloc[row-1] != "None" and df['B'].iloc[row-1] != "None":
            if df['A'].iloc[row-1] == "cooperate" and df['B'].iloc[row-1] == "defect":
                opp_betray_count += 1
                if df['A'].iloc[row] == "defect":
                    r_count += 1
        else:
            r_invalid +=1

    if opp_betray_count == 0:
        r_percentage = -1
    else:
        r_percentage = r_count / opp_betray_count

    #Troublemaking
    t_count = 0
    opportunity_count = 0
    t_invalid = 0
    for row in range(0,df.shape[0]):
        if row == 0:
            if df['A'].iloc[row] != "None":
                opportunity_count += 1
                if df['A'].iloc[row] == "defect":
                    t_count +=1
            else:
                t_invalid +=1 
        else:        
            if df['A'].iloc[row] != "None" and df['B'].iloc[row-1] != "None":
                if df['B'].iloc[row-1] == "cooperate":
                    opportunity_count += 1
                    if df['A'].iloc[row] == "defect":
                        t_count += 1
            else:
                t_invalid +=1
                
    if opportunity_count == 0:
        t_percentage = -1
    else:
        t_percentage = t_count / opportunity_count

    if df2 is not None:
        #Lying
        l_count = 0
        opportunity_count = 0
        l_invalid = 0
        for row in range(0,df.shape[0]):
            if df['A'].iloc[row] != df2['A'].iloc[row]:
                opportunity_count += 1
                l_count += 1
            else:
                opportunity_count += 1
                    
        if opportunity_count == 0:
            l_percentage = -1
        else:
            l_percentage = l_count / opportunity_count
        #betrayal
        b_count = 0
        opportunity_count = 0
        b_invalid = 0
        for row in range(0,df.shape[0]):
            if df2['B'].iloc[row]=="cooperate":
                if df['A'].iloc[row] == "defect" and df2['A'].iloc[row]=="cooperate":
                    opportunity_count += 1
                    b_count += 1
                else:
                    opportunity_count += 1
                    
        if opportunity_count == 0:
            b_percentage = -1
        else:
            b_percentage = b_count / opportunity_count
        #manipulative
        m_count = 0
        opportunity_count = 0
        m_invalid = 0
        for row in range(0,df.shape[0]):
            if df2['B'].iloc[row]=="defect":
                if df['A'].iloc[row] == "defect" and df2['A'].iloc[row]=="cooperate":
                    opportunity_count += 1
                    m_count += 1
                else:
                    opportunity_count += 1
                    
        if opportunity_count == 0:
            m_percentage = -1
        else:
            m_percentage = m_count / opportunity_count
        #trusting
        tr_count = 0
        opportunity_count = 0
        tr_invalid = 0
        for row in range(0,df.shape[0]):
            if df2['B'].iloc[row]=="cooperate":
                if df['A'].iloc[row] == "cooperate" and df2['A'].iloc[row]=="cooperate":
                    opportunity_count += 1
                    tr_count += 1
                else:
                    opportunity_count += 1
                    
        if opportunity_count == 0:
            tr_percentage = -1
        else:
            tr_percentage = tr_count / opportunity_count
    else:
        l_percentage = -1
        m_percentage = -1
        b_percentage = -1
        tr_percentage = -1
        
    #SFEM
    sfem = SFEM(df)
    sfem_res = sfem.save_results()
    
    # Return the result as a dictionary
    results = {
        'defect_rate': defect_rate,
        'reciprocity_rate': rr_percentage,
        'reciprocity_invalid': rr_invalid,
        'forgiveness_rate': f_percentage,
        'forgiveness_invalid': f_invalid,
        'exploitability_rate': e_percentage,
        'explotability_invalid': e_invalid,
        'retaliatory_rate': r_percentage,
        'rectaliatory_invalid': r_invalid,
        'troublemaking_rate': t_percentage,
        'troublemaking_invalid': t_invalid,    
        'lying_rate': l_percentage,
        'betrayal_rate': b_percentage,
        'manipulative_rate': m_percentage,
        'trusting_rate': tr_percentage,
        'score': score,
        'total_score': total_score,
        'sfem': sfem_res    
	}
    
    return results

In [ ]:
strategies = ['s01_allc', 's02_alld', 's03_tft', 's04_dtft', 's05_tf2t', 's06_tf3t', 's07_2tft', 's08_2tf2t', 's09_t2', 's10_grim', 's11_grim2', 's12_grim3', 's13_wsls', 's14_2wsls', 's15_ctod', 's16_dtf2t', 's17_dtf3t', 's18_dgrim2', 's19_dgrim3', 's20_dcalt']

In [ ]:
for exp in glob.glob("output/*llmvllm*"):
    print(exp)
    results = []
    for i in range(25):
        with open(os.path.join(exp,"prisoner_A_log_history_iteration_{}.json".format(i)), "r") as f:
            j = json.load(f)
        df = pd.DataFrame(j)
        df.columns = ["A", "B"]
        with open(os.path.join(exp,"prisoner_A_log_history_iteration_comms_{}.json".format(i)), "r") as f:
            j = json.load(f)
        df2 = pd.DataFrame(j)
        df2.columns = ["A", "B"]
        res = metrics(df, df2)
        for strat in strategies:
            res[strat] = res['sfem'].loc[strat]["Estimates"]
        results.append(res)
    pd.DataFrame(results).to_csv(os.path.join(exp,"results.csv"))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

opponents = ["-AC*"]

for opp in opponents:

    summary = pd.DataFrame()
    
    fig, ax = plt.subplots(1,1, figsize=(10, 6))

    for exp in glob.glob(f"output/*baseline{opp}"):
        exp_res = os.path.join(exp, "results.csv")
        df = pd.read_csv(exp_res)

        df_metrics = pd.DataFrame()
        metrics = ["troublemaking_rate"]
        for m in metrics:
            df_metrics[m] = df[m]

        df_metrics.replace(-1, np.nan, inplace=True)
        mask = ~np.isnan(df_metrics)
        filtered_data = [df_metrics[d][mask[m]] for d, m in zip(df_metrics, mask)]
        filtered_data_ = []
        for f in filtered_data:
            if not f.any():
                f = [0]
            filtered_data_.append(f)
        
        summary[exp]=filtered_data_[0]

    exp_list = glob.glob(f"output/*{opp}")
    exp_list.remove(exp)
    exp_list.sort()

    for i, exp in enumerate(exp_list):
        exp_res = os.path.join(exp, "results.csv")
        df = pd.read_csv(exp_res)

        df_metrics = pd.DataFrame()
        metrics = ["troublemaking_rate"]
        for m in metrics:
            df_metrics[m] = df[m]

        df_metrics.replace(-1, np.nan, inplace=True)
        mask = ~np.isnan(df_metrics)
        filtered_data = [df_metrics[d][mask[m]] for d, m in zip(df_metrics, mask)]
        filtered_data_ = []
        for f in filtered_data:
            if not f.any():
                f = [0]
            filtered_data_.append(f)

        summary[exp]=filtered_data_[0]
    # fig.savefig(f"{opp}.png")
    x_labels = []
    for col in summary.columns:
        col = col.strip("output/")
        x_labels.append(col)

    mask = ~np.isnan(summary)
    filtered_data = [summary[d][mask[m]] for d, m in zip(summary, mask)]
    filtered_data_ = []
    for f in filtered_data:
        if not f.any():
            f = [0]
        filtered_data_.append(f)
    
    ax.boxplot(filtered_data_)
    ax.set_title("Troublemaking rate")
    ax.set_ylim(0,1)
    ax.set_xticks(range(1,22), labels=x_labels)
    ax.tick_params(axis='x', labelrotation=90)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# metrics = ["defect_rate", "reciprocity_rate", "forgiveness_rate", "exploitability_rate", "retaliatory_rate", "troublemaking_rate"]
opponents = ["-RND-comms-selfish"]

for opp in opponents:

    summary = pd.DataFrame()
    
    fig, ax = plt.subplots(1,1, figsize=(10, 6))

    for exp in glob.glob(f"output/*baseline{opp}"):
        exp_res = os.path.join(exp, "results.csv")
        df = pd.read_csv(exp_res)

        df_metrics = pd.DataFrame()
        metrics = ["lying_rate"]
        for m in metrics:
            df_metrics[m] = df[m]

        df_metrics.replace(-1, np.nan, inplace=True)
        mask = ~np.isnan(df_metrics)
        filtered_data = [df_metrics[d][mask[m]] for d, m in zip(df_metrics, mask)]
        filtered_data_ = []
        for f in filtered_data:
            if not f.any():
                f = [0]
            filtered_data_.append(f)
        
        summary[exp]=filtered_data_[0]

    exp_list = glob.glob(f"output/*{opp}")
    exp_list.remove(exp)
    exp_list.sort()

    for i, exp in enumerate(exp_list):
        exp_res = os.path.join(exp, "results.csv")
        df = pd.read_csv(exp_res)

        df_metrics = pd.DataFrame()
        metrics = ["lying_rate"]
        for m in metrics:
            df_metrics[m] = df[m]

        df_metrics.replace(-1, np.nan, inplace=True)
        mask = ~np.isnan(df_metrics)
        filtered_data = [df_metrics[d][mask[m]] for d, m in zip(df_metrics, mask)]
        filtered_data_ = []
        for f in filtered_data:
            if not f.any():
                f = [0]
            filtered_data_.append(f)

        summary[exp]=filtered_data_[0]
    # fig.savefig(f"{opp}.png")
    x_labels = []
    for col in summary.columns:
        col = col.strip("output/")
        # col = col.strip(opp)
        x_labels.append(col)
    
    ax.boxplot(summary)
    ax.set_title("Lying rate")
    ax.set_ylim(0,1)
    ax.set_xticks(range(1,12), labels=x_labels)
    ax.tick_params(axis='x', labelrotation=90)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# metrics = ["defect_rate", "reciprocity_rate", "forgiveness_rate", "exploitability_rate", "retaliatory_rate", "troublemaking_rate"]
opponents = ["-URND2p", "-RND2", "-URND2p03"]

summary = pd.DataFrame()
fig, ax = plt.subplots(1,1, figsize=(10, 6))

for opp in opponents:
    for exp in glob.glob(f"output/*baseline{opp}"):
        exp_res = os.path.join(exp, "results.csv")
        df = pd.read_csv(exp_res)

        df_metrics = pd.DataFrame()
        metrics = ["forgiveness_rate"]
        for m in metrics:
            df_metrics[m] = df[m]

        df_metrics.replace(-1, np.nan, inplace=True)
        mask = ~np.isnan(df_metrics)
        filtered_data = [df_metrics[d][mask[m]] for d, m in zip(df_metrics, mask)]
        filtered_data_ = []
        for f in filtered_data:
            if not f.any():
                f = [0]
            filtered_data_.append(f)
        
        summary[exp]=filtered_data_[0]

    exp_list = glob.glob(f"output/*{opp}")
    exp_list.remove(exp)
    exp_list.sort()

    for i, exp in enumerate(exp_list):
        exp_res = os.path.join(exp, "results.csv")
        df = pd.read_csv(exp_res)

        df_metrics = pd.DataFrame()
        metrics = ["forgiveness_rate"]
        for m in metrics:
            df_metrics[m] = df[m]

        df_metrics.replace(-1, np.nan, inplace=True)
        mask = ~np.isnan(df_metrics)
        filtered_data = [df_metrics[d][mask[m]] for d, m in zip(df_metrics, mask)]
        filtered_data_ = []
        for f in filtered_data:
            if not f.any():
                f = [0]
            filtered_data_.append(f)

        summary[exp]=filtered_data_[0]
    # fig.savefig(f"{opp}.png")

summary_joined = pd.DataFrame()

ds = summary.filter(regex=r'(?i)baseline')
baseline = []
for d in ds:
    baseline.extend(summary[d])
summary_joined['baseline'] = baseline

experiments = ["agreeableness-minus", "agreeableness-plus","conscientiousness-minus", "conscientiousness-plus","extraversion-minus", "extraversion-plus","neuroticism-minus", "neuroticism-plus","openness-minus", "openness-plus",]
for exp in experiments:
    ds = summary.filter(regex=r'(?i){}'.format(exp))
    temp = []
    for d in ds:
        temp.extend(summary[d])
    summary_joined[exp] = temp

x_labels = []
for col in summary_joined.columns:
    # col = col.strip("output/")
    # col = col.strip(opp)
    x_labels.append(col)
    
mask = ~np.isnan(summary_joined)
filtered_data = [summary_joined[d][mask[m]] for d, m in zip(summary_joined, mask)]
filtered_data_ = []
for f in filtered_data:
    if not f.any():
        f = [0]
    filtered_data_.append(f)
    
ax.boxplot(filtered_data_)
ax.set_title("Forgiveness rate")
ax.set_ylim(0,1)
ax.set_xticks(range(1,12), labels=x_labels)
ax.tick_params(axis='x', labelrotation=90)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# metrics = ["defect_rate", "reciprocity_rate", "forgiveness_rate", "exploitability_rate", "retaliatory_rate", "troublemaking_rate"]
opponents = ["*llmvllm*"]

for opp in opponents:
    summary = pd.DataFrame()
    exp_list = glob.glob(f"output/{opp}")
    # exp_list.remove(exp)
    exp_list.sort()

    for i, exp in enumerate(exp_list):
        exp_res = os.path.join(exp, "results.csv")
        df = pd.read_csv(exp_res)

        df_metrics = pd.DataFrame()
        metrics = ["score","total_score"]
        for m in metrics:
            df_metrics[m] = df[m]

        summary[exp] = -1*df_metrics['total_score'] + 2*df_metrics['score']

In [ ]:
experiments = ["baseline", "agreeableness-plus","agreeableness-minus", "conscientiousness-plus","conscientiousness-minus","extraversion-plus","extraversion-minus","neuroticism-plus","neuroticism-minus","openness-plus","openness-minus"]

summary_mean = summary.mean()
summary_mean

import seaborn as sns

scores = pd.DataFrame()

for e1 in experiments:
    scores[e1] = -1
    for e2 in experiments:
        scores.loc[e2] = -1

for run in summary_mean.index:
    run_name = run.split("_")[0].split("/")[-1]
    splits = run_name.split("-")
    if splits[2] == "baseline":
        p1 = "baseline"
        if splits[3] == "baseline":
            p2 = "baseline"
        else: 
            p2 = "-".join(splits[3:5])
    else:
        p1 = "-".join(splits[2:4])
        if splits[4] == "baseline":
            p2 = "baseline"
        else: 
            p2 = "-".join(splits[4:6])
    scores.loc[p1,p2] = summary_mean[run]

scores

In [ ]:
import seaborn as sns

fig, ax = plt.subplots(1,1)
ax = sns.heatmap(scores, annot=True)
ax.set_title("Total score")
ax.set_ylabel("Prisoner A")
ax.set_xlabel("Prisoner B")

In [ ]:
import seaborn as sns

fig, ax = plt.subplots(1,1)
ax = sns.heatmap(scores, annot=True)
ax.set_title("Personal score (A) difference")
ax.set_ylabel("Prisoner A")
ax.set_xlabel("Prisoner B")